## Optimizer

참고 : http://shuuki4.github.io/deep%20learning/2016/05/20/Gradient-Descent-Algorithm-Overview.html

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.autograd import Variable

from torchvision import transforms
from torchvision import datasets

batch_size = 4
learning_rate = 0.01
num_epochs = 5

transform = transforms.ToTensor()

# Data Loader
trainData = datasets.CIFAR10(root="./data",train=True,transform=transform,download=True)
trainLoader = DataLoader(trainData,batch_size=batch_size,shuffle=True)

testData = datasets.CIFAR10(root="./data", train=False, transform=transform)
testLoader = DataLoader(testData, batch_size=batch_size, shuffle=False)

In [ ]:
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        
        return out

In [ ]:
# Model
model = CNN()

# Loss
criterion = nn.CrossEntropyLoss()

# Optimizer
SGD = torch.optim.SGD(model.parameters(), lr=learning_rate)

Adagrad = torch.optim.Adagrad(model.parameters(), lr=0.01, lr_decay=0, weight_decay=0, initial_accumulator_value=0)

Adadelta = torch.optim.Adadelta(params, lr=1.0, rho=0.9, eps=1e-06, weight_decay=0)

RMSprop = torch.optim.RMSprop(params, lr=0.01, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0, centered=False)

Adam = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

LBFGS = torch.optim.LBFGS(params, lr=1, max_iter=20, max_eval=None, tolerance_grad=1e-05, tolerance_change=1e-09, history_size=100, line_search_fn=None)

## VGG16

![VGG16](https://qph.fs.quoracdn.net/main-qimg-e657c195fc2696c7d5fc0b1e3682fde6)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.autograd import Variable

from torchvision import transforms
from torchvision import datasets

batch_size = 4
learning_rate = 0.01
num_epochs = 5

transform = transforms.ToTensor()

# Data Loader
trainData = datasets.CIFAR10(root="./data",train=True,transform=transform,download=True)
trainLoader = DataLoader(trainData,batch_size=batch_size,shuffle=True)

testData = datasets.CIFAR10(root="./data", train=False, transform=transform)
testLoader = DataLoader(testData, batch_size=batch_size, shuffle=False)

Files already downloaded and verified


In [3]:
def conv(c_in, c_out, k_size, stride=1, pad=1, bn=True):
    """Custom convolutional layer for simplicity."""
    layers = []
    layers.append(nn.Conv2d(c_in, c_out, k_size, stride, pad, bias=False))
    if bn:
        layers.append(nn.BatchNorm2d(c_out))
    layers.append(nn.ReLU())
    return nn.Sequential(*layers)

In [4]:
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()

        self.layer1 = nn.Sequential(
            conv(3, 64, 3),
            conv(64, 64, 3),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = nn.Sequential(
            conv(64, 128, 3),
            conv(128, 128, 3),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer3 = nn.Sequential(
            conv(128, 256, 3),
            conv(256, 256, 3),
            conv(256, 256, 3),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer4 = nn.Sequential(
            conv(256, 512, 3),
            conv(512, 512, 3),
            conv(512, 512, 3),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer5 = nn.Sequential(
            conv(512, 512, 3),
            conv(512, 512, 3),
            conv(512, 512, 3),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.FC1 = nn.Sequential(
            nn.Linear(512, 4096),
            nn.BatchNorm1d(4096),
            nn.ReLU())

        self.FC2 = nn.Sequential(
            nn.Linear(4096, 4096),
            nn.BatchNorm1d(4096),
            nn.ReLU())

        self.FC3 = nn.Sequential(
            nn.Linear(4096, 10),
            nn.BatchNorm1d(10),
            nn.Softmax())

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)

        out = out.view(out.size(0), -1)

        out = self.FC1(out)
        out = self.FC2(out)
        out = self.FC3(out)

        return out

In [6]:
vgg16 = VGG16()

# Loss Function
Loss_function = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.Adam(vgg16.parameters(), lr=learning_rate)


In [10]:
# Train the model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainLoader):
        optimizer.zero_grad()
        outputs = vgg16(images)
        loss = Loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print('Epoch [%d/%d], Iter [%d/%d] Loss %.4f'
                  % (epoch+1, num_epochs, i, len(trainLoader), loss.item()))

C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\container.py:91: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch [1/5], Iter [0/12500] Loss 2.2575


KeyboardInterrupt: 

In [ ]:
# Test the model
vgg16.eval()
correct = 0
total = 0

for images, labels in testLoader:
    images = Variable(images).cuda()
    outputs = vgg16(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum()

print('Test Accuracy : %d %%' % (100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model_nn.ckpt')

## 과제. VGG19 구현하기

![VGG19](https://qcloud.coding.net/u/vincentqin/p/blogResource/git/raw/master/CNN-Architectures/89475867.png)